# Real-Time Prediction with MTA Bus Time GTFS-Realtime Feed

This notebook demonstrates a real-time data pipeline using the MTA Bus Time GTFS-Realtime feed. The pipeline ingests, processes, stores, queries, and visualizes bus data using Kafka, Flink, Iceberg, Trino, and Superset.

## 1. Setup and Install Dependencies

This project uses a `pyproject.toml` file for dependency management. Make sure to install the dependencies using `pip` or `uv sync` before running this notebook.

## 2. Access and Fetch GTFS-Realtime Data

Use the MTA GTFS-Realtime API to fetch vehicle positions, trip updates, and alerts. Parse the protobuf data into a readable format.

In [1]:
import requests
from google.protobuf import json_format
from google.transit import gtfs_realtime_pb2
import os
from dotenv import load_dotenv

# Load API key from .env file if available
load_dotenv()
API_KEY = os.getenv("MTA_API_KEY", "YOUR_MTA_API_KEY")

# Example: Fetch vehicle positions from MTA Bus Time GTFS-Realtime API
VEHICLE_POSITIONS_URL = f"http://gtfsrt.prod.obanyc.com/vehiclePositions?key={API_KEY}"

try:
    response = requests.get(VEHICLE_POSITIONS_URL)
    if response.status_code == 200:
        feed = gtfs_realtime_pb2.FeedMessage()
        feed.ParseFromString(response.content)
        
        # Convert protobuf to JSON
        vehicle_positions = [json_format.MessageToDict(entity) for entity in feed.entity]
        print(f"Successfully fetched {len(vehicle_positions)} vehicle positions")
        print(vehicle_positions[:2] if vehicle_positions else "No data available")  # Preview first two entries
    else:
        print(f"Error: {response.status_code} - {response.text}")
        print("Note: You may need to set up a valid MTA API key in a .env file")
except Exception as e:
    print(f"Error fetching data: {e}")
    print("\nNOTE: If you're getting module import errors, please run the first cell to install dependencies.")

Successfully fetched 2419 vehicle positions
[{'id': 'MTA NYCT_9770', 'vehicle': {'trip': {'tripId': 'MV_B5-Weekday-SDon-042000_M116_X01', 'startDate': '20250501', 'routeId': 'M116', 'directionId': 1}, 'position': {'latitude': 40.79613, 'longitude': -73.93236, 'bearing': 233.86462}, 'timestamp': '1746095839', 'stopId': '405315', 'vehicle': {'id': 'MTA NYCT_9770'}}}, {'id': 'MTA NYCT_8444', 'vehicle': {'trip': {'tripId': 'JA_B5-Weekday-SDon-037000_Q30_402', 'startDate': '20250501', 'routeId': 'Q30', 'directionId': 0}, 'position': {'latitude': 40.74045, 'longitude': -73.78713, 'bearing': 14.370002}, 'timestamp': '1746095838', 'stopId': '501444', 'vehicle': {'id': 'MTA NYCT_8444'}}}]


## Setup Docker Compose for Kafka, Flink, and Trino

Creating a Docker Compose setup for local development of the pipeline components.

In [2]:
%%writefile docker-compose.yml
services:
  # Zookeeper for Kafka
  zookeeper:
    image: confluentinc/cp-zookeeper:7.4.0
    hostname: zookeeper
    container_name: zookeeper
    ports:
      - "2181:2181"
    environment:
      ZOOKEEPER_CLIENT_PORT: 2181
      ZOOKEEPER_TICK_TIME: 2000

  # Kafka broker
  kafka:
    image: confluentinc/cp-kafka:7.4.0
    hostname: kafka
    container_name: kafka
    depends_on:
      - zookeeper
    ports:
      - "9092:9092"
      - "29092:29092"
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: zookeeper:2181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka:9092,PLAINTEXT_HOST://localhost:29092
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: PLAINTEXT:PLAINTEXT,PLAINTEXT_HOST:PLAINTEXT
      KAFKA_INTER_BROKER_LISTENER_NAME: PLAINTEXT
      KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR: 1
      KAFKA_GROUP_INITIAL_REBALANCE_DELAY_MS: 0

  # Kafka Control Center
  control-center:
    image: confluentinc/cp-enterprise-control-center:7.4.0
    hostname: control-center
    container_name: control-center
    depends_on:
      - kafka
    ports:
      - "9021:9021"
    environment:
      CONTROL_CENTER_BOOTSTRAP_SERVERS: kafka:9092
      CONTROL_CENTER_REPLICATION_FACTOR: 1
      PORT: 9021

  # Flink JobManager
  flink-jobmanager:
    image: apache/flink:1.18
    hostname: jobmanager
    container_name: flink-jobmanager
    ports:
      - "8081:8081"
    command: jobmanager
    environment:
      - |  
        FLINK_PROPERTIES=
        jobmanager.rpc.address: jobmanager        
        parallelism.default: 1

  # Flink TaskManager
  flink-taskmanager:
    image: apache/flink:1.18
    hostname: taskmanager
    container_name: flink-taskmanager
    depends_on:
      - flink-jobmanager
    command: taskmanager
    environment:
      - |  
        FLINK_PROPERTIES=
        jobmanager.rpc.address: jobmanager        
        taskmanager.numberOfTaskSlots: 2
        parallelism.default: 1

  # MinIO (S3-compatible storage for Iceberg)
  minio:
    image: minio/minio
    container_name: minio
    ports:
      - "9000:9000"
      - "9001:9001"
    environment:
      MINIO_ROOT_USER: minio
      MINIO_ROOT_PASSWORD: minio123
    command: server /data --console-address ":9001"
    volumes:
      - minio-data:/data

  # Trino coordinator
  trino-coordinator:
    image: trinodb/trino:413
    container_name: trino-coordinator
    ports:
      - "8080:8080"
    volumes:
      - ./trino/catalog:/etc/trino/catalog
      - ./trino/etc:/etc/trino/etc

volumes:
  minio-data:

Writing docker-compose.yml


## 3. Publish Data to Kafka

Create Kafka topics and write a Python producer to publish GTFS-Realtime data to Kafka topics such as `vehicle_positions`, `trip_updates`, and `alerts`.

In [ ]:
from confluent_kafka import Producer
import json
import time

# Update Kafka broker address based on Docker Compose configuration
KAFKA_BROKER = 'localhost:29092'  # External port for local access
TOPIC = 'vehicle_positions'

def create_kafka_producer():
    try:
        producer_config = {
            'bootstrap.servers': KAFKA_BROKER,
            'client.id': 'gtfs-realtime-producer'
        }
        producer = Producer(producer_config)
        print(f"Successfully connected to Kafka broker at {KAFKA_BROKER}")
        return producer
    except Exception as e:
        print(f"Error creating Kafka producer: {e}")
        print("\nNOTE: Make sure Docker Compose is running with Kafka services.")
        return None

def delivery_report(err, msg):
    if err is not None:
        print(f"Delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

# Only run if we have vehicle positions data
if 'vehicle_positions' in locals() and vehicle_positions:
    producer = create_kafka_producer()
    if producer:
        try:
            # Publish vehicle positions to Kafka
            print(f"Publishing {len(vehicle_positions)} records to Kafka topic: {TOPIC}")
            for vp in vehicle_positions:
                producer.produce(TOPIC, json.dumps(vp).encode('utf-8'), callback=delivery_report)
                producer.poll(0)  # Trigger callbacks
            
            producer.flush()
            print("All messages published to Kafka")
        except Exception as e:
            print(f"Error publishing to Kafka: {e}")
else:
    print("No vehicle position data available to publish to Kafka.")
    print("Run the previous cell to fetch GTFS-Realtime data first.")

## 4. Stream Processing with Apache Flink

Set up a Flink job to consume data from Kafka, process it (e.g., calculate headway variance, detect bus bunching), and write the results to Iceberg tables.

In [ ]:
# Example: Flink job setup for processing GTFS-Realtime data
# This would be implemented in a separate Flink application

%%writefile flink_job_example.py
# This is a sample PyFlink job to process vehicle position data
# You would submit this to the Flink cluster we set up in Docker Compose

from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment, EnvironmentSettings

def run_flink_job():
    # Set up the execution environment
    env = StreamExecutionEnvironment.get_execution_environment()
    env_settings = EnvironmentSettings.Builder().build()
    table_env = StreamTableEnvironment.create(env, environment_settings=env_settings)
    
    # Define Kafka source
    table_env.execute_sql("""
    CREATE TABLE vehicle_positions (
        vehicle_id STRING,
        trip_id STRING,
        route_id STRING,
        timestamp BIGINT,
        latitude DOUBLE,
        longitude DOUBLE,
        speed DOUBLE,
        bearing DOUBLE,
        event_time AS TO_TIMESTAMP(FROM_UNIXTIME(timestamp / 1000)),
        WATERMARK FOR event_time AS event_time - INTERVAL '5' SECONDS
    ) WITH (
        'connector' = 'kafka',
        'topic' = 'vehicle_positions',
        'properties.bootstrap.servers' = 'kafka:9092',
        'properties.group.id' = 'flink-vehicle-analyzer',
        'format' = 'json',
        'scan.startup.mode' = 'latest-offset'
    )
    """)
    
    # Define Iceberg sink for processed data
    table_env.execute_sql("""
    CREATE TABLE vehicle_metrics (
        route_id STRING,
        window_start TIMESTAMP(3),
        window_end TIMESTAMP(3),
        vehicle_count BIGINT,
        avg_speed DOUBLE,
        PRIMARY KEY (route_id, window_start) NOT ENFORCED
    ) WITH (
        'connector' = 'iceberg',
        'catalog-name' = 'iceberg_catalog',
        'catalog-type' = 'hive',
        'warehouse' = 's3a://warehouse/iceberg',
        'format-version' = '2'
    )
    """)
    
    # Calculate metrics with a window aggregation
    table_env.execute_sql("""
    INSERT INTO vehicle_metrics
    SELECT
        route_id,
        window_start,
        window_end,
        COUNT(DISTINCT vehicle_id) AS vehicle_count,
        AVG(speed) AS avg_speed
    FROM TABLE(
        TUMBLE(TABLE vehicle_positions, DESCRIPTOR(event_time), INTERVAL '1' MINUTE)
    )
    GROUP BY route_id, window_start, window_end
    """)

if __name__ == '__main__':
    run_flink_job()

print("Sample Flink job code has been generated to flink_job_example.py")
print("NOTE: To run this job, you would need to submit it to the Flink cluster")
print("using the Flink CLI or REST API.")

## 5. Store Processed Data in Iceberg

Configure Iceberg to store processed data in a time-partitioned format. Define schemas for raw and aggregated data tables.

In [ ]:
# Create Trino catalog configuration for Iceberg tables
!mkdir -p trino/catalog

%%writefile trino/catalog/iceberg.properties
connector.name=iceberg
hive.metastore.uri=thrift://hive-metastore:9083
iceberg.catalog.type=hive_metastore
iceberg.file-format=PARQUET
hive.s3.endpoint=http://minio:9000
hive.s3.path-style-access=true
hive.s3.aws-access-key=minio
hive.s3.aws-secret-key=minio123

# Example: Iceberg table schema definition
print("""Example Iceberg table creation in Trino:""")
print("""CREATE SCHEMA IF NOT EXISTS iceberg.mta_data;

CREATE TABLE iceberg.mta_data.vehicle_positions (
    vehicle_id VARCHAR,
    trip_id VARCHAR,
    route_id VARCHAR,
    timestamp TIMESTAMP,
    latitude DOUBLE,
    longitude DOUBLE,
    speed DOUBLE,
    bearing DOUBLE
) WITH (
    format = 'PARQUET',
    partitioning = ARRAY['day(timestamp)']
);
""")

## 6. Query Data with Trino

Use Trino to query the Iceberg tables for real-time and historical analytics. Write SQL queries to retrieve insights such as bus delays and bunching events.

In [ ]:
# Example: Query with Trino using Python client
print("NOTE: This requires Trino to be running properly. The code below shows how to query.")
print("Install the client with: pip install trino\n")

import warnings
warnings.filterwarnings('ignore')

trino_example_code = """
import trino
import pandas as pd

def query_trino():
    # Connect to Trino
    conn = trino.dbapi.connect(
        host='localhost',
        port=8080,
        user='trino',
        catalog='iceberg',
        schema='mta_data',
    )
    
    # Example query
    query = """
        SELECT 
            route_id,
            date_trunc('hour', timestamp) as hour,
            count(*) as position_count,
            avg(speed) as avg_speed,
            min(speed) as min_speed,
            max(speed) as max_speed
        FROM vehicle_positions
        WHERE timestamp >= current_timestamp - interval '1' day
        GROUP BY 1, 2
        ORDER BY 1, 2
        LIMIT 10
    """
    
    # Execute query and convert to DataFrame
    cur = conn.cursor()
    cur.execute(query)
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(cur.fetchall(), columns=columns)
    return df

# Run query and display results
try:
    results_df = query_trino()
    print(results_df)
except Exception as e:
    print(f"Error querying Trino: {e}")
"""

print(trino_example_code)

## 7. Visualize Real-Time Data in Superset

Create a Superset dashboard to visualize real-time bus locations, headway variance, and alerts. Include auto-refreshing charts and maps.

**Instructions for Setting Up Superset:**

1. Connect Superset to your Trino/Iceberg backend.
2. Add the `vehicle_positions` and other relevant tables as datasets.
3. Create charts (e.g., map, time series) to visualize bus locations and metrics.
4. Build a dashboard and enable auto-refresh for real-time updates.

For local development, you can add Superset to your Docker Compose file with:

```yaml
  superset:
    image: apache/superset:latest
    container_name: superset
    ports:
      - "8088:8088"
    environment:
      - SUPERSET_SECRET_KEY=your_secret_key
```

# Building a Modern Data Engineering Portfolio: Local-First vs Cloud-First Pipeline

This section demonstrates how to build a comprehensive data engineering project using both local-first (DuckDB/MotherDuck) and cloud-first (AWS, GCP, Azure) approaches. We'll use the NYC Taxi Trips dataset as an example, and walk through ingestion, transformation, storage, querying, and portfolio presentation best practices.

## Step 1: Project Setup and Data Ingestion

- Install Python 3, DuckDB, and optionally Docker for local development.
- Set up cloud resources (S3, GCS, ADLS) and CLIs for AWS, GCP, Azure.
- Download sample NYC Taxi data (yellow/green trip records) as CSV files.
- Use ELT: Extract (download), Load (to DuckDB/cloud), Transform (later).

In [ ]:
# Create directories for data
!mkdir -p data/raw data/parquet

# Download sample data (NYC Yellow Taxi, Jan 2021)
!wget -nc https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz -P data/raw/ -q
print("Downloaded NYC Taxi data to data/raw/yellow_tripdata_2021-01.csv.gz")

## Step 2: Local Data Storage with DuckDB/MotherDuck

- Create a DuckDB database and load CSV data.
- Optionally, convert to Parquet for interoperability.
- (Optional) Sync to MotherDuck for cloud access.

In [ ]:
import duckdb
import os

db_path = 'nyc_taxi.duckdb'
data_file = 'data/raw/yellow_tripdata_2021-01.csv.gz'

# Check if the data file exists
if os.path.exists(data_file):
    # Connect to DuckDB
    con = duckdb.connect(db_path)
    
    # Create yellow_trips_raw table if it doesn't exist
    if len(con.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='yellow_trips_raw'").fetchall()) == 0:
        print(f"Creating yellow_trips_raw table from {data_file}...")
        con.execute(f"CREATE TABLE yellow_trips_raw AS SELECT * FROM read_csv_auto('{data_file}');")
        print(f"Table created with {con.execute('SELECT COUNT(*) FROM yellow_trips_raw').fetchone()[0]} rows")
    else:
        print("Table yellow_trips_raw already exists")
        print(f"It contains {con.execute('SELECT COUNT(*) FROM yellow_trips_raw').fetchone()[0]} rows")
    
    # Export sample to Parquet (if it doesn't exist)
    parquet_file = 'data/parquet/yellow_tripdata_2021-01.parquet'
    if not os.path.exists(parquet_file):
        print(f"Exporting to Parquet: {parquet_file}")
        con.execute(f"COPY (SELECT * FROM yellow_trips_raw) TO '{parquet_file}' (FORMAT PARQUET, COMPRESSION ZSTD);")
        print("Export complete")
    else:
        print(f"Parquet file already exists: {parquet_file}")
    
    # Show schema and sample data
    print("\nTable Schema:")
    print(con.execute("DESCRIBE yellow_trips_raw").fetchdf())
    
    print("\nSample Data:")
    print(con.execute("SELECT * FROM yellow_trips_raw LIMIT 5").fetchdf())
    
    con.close()
else:
    print(f"Data file not found: {data_file}")
    print("Please run the previous cell to download the data first.")

## Step 3: Cloud Data Storage & Querying Setup

- AWS: Upload Parquet to S3, define Athena table.
- GCP: Upload to GCS, load into BigQuery or create external table.
- Azure: Upload to ADLS, create Synapse external table.

In [ ]:
import boto3
import os
from dotenv import load_dotenv

# Load environment variables for AWS credentials
load_dotenv()

def upload_to_s3(local_file, bucket_name, s3_key):
    print(f"This function would upload {local_file} to s3://{bucket_name}/{s3_key}")
    print("To actually run this, you need to:")
    print("1. Set AWS credentials in a .env file or environment variables")
    print("2. Create the S3 bucket")
    print("3. Uncomment the code below")
    
    """
    # Create an S3 client
    s3_client = boto3.client(
        's3',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        region_name=os.getenv('AWS_REGION', 'us-east-1')
    )
    
    # Upload the file
    try:
        s3_client.upload_file(local_file, bucket_name, s3_key)
        return True
    except Exception as e:
        print(f"Error uploading to S3: {e}")
        return False
    """
    return True

# Example usage
parquet_file = 'data/parquet/yellow_tripdata_2021-01.parquet'
bucket_name = 'my-taxi-data-bucket'
s3_key = 'tripdata/yellow_tripdata_2021-01.parquet'

if os.path.exists(parquet_file):
    if upload_to_s3(parquet_file, bucket_name, s3_key):
        print(f"\nTo query this data in AWS Athena, you would create an external table with:")
        print("""
        CREATE EXTERNAL TABLE yellow_trips (
          VendorID INT,
          tpep_pickup_datetime TIMESTAMP,
          tpep_dropoff_datetime TIMESTAMP,
          passenger_count INT,
          trip_distance DOUBLE,
          ...
        )
        STORED AS PARQUET
        LOCATION 's3://my-taxi-data-bucket/tripdata/'
        TBLPROPERTIES ('parquet.compression'='SNAPPY');
        """)
else:
    print(f"Parquet file not found: {parquet_file}")
    print("Please run the previous steps to create the Parquet file.")

## Step 4: Data Transformation and Analytics

- Use DuckDB or dbt for local SQL transformations.
- Use Athena/BigQuery/Synapse SQL for cloud.
- Example: Clean, filter, and aggregate taxi trip data.

In [ ]:
import duckdb

# Connect to the database
con = duckdb.connect('nyc_taxi.duckdb')

# Check if the raw table exists
if len(con.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='yellow_trips_raw'").fetchall()) > 0:
    # Create a cleaned table with transformations
    if len(con.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='yellow_trips_cleaned'").fetchall()) == 0:
        print("Creating cleaned table with transformations...")
        
        # Note: Table columns might vary, adjust SQL as needed
        try:
            con.execute("""
            CREATE TABLE yellow_trips_cleaned AS
            SELECT
              VendorID,
              tpep_pickup_datetime::TIMESTAMP as pickup_datetime,
              tpep_dropoff_datetime::TIMESTAMP as dropoff_datetime,
              passenger_count,
              trip_distance,
              RatecodeID,
              PULocationID as pickup_location_id,
              DOLocationID as dropoff_location_id,
              payment_type,
              fare_amount,
              tip_amount,
              total_amount,
              EXTRACT(HOUR FROM tpep_pickup_datetime::TIMESTAMP) as pickup_hour,
              DATE_TRUNC('day', tpep_pickup_datetime::TIMESTAMP) as pickup_date
            FROM yellow_trips_raw
            WHERE trip_distance > 0 
              AND fare_amount > 0
              AND passenger_count > 0
              AND tpep_dropoff_datetime > tpep_pickup_datetime;
            """)
            
            # Create an aggregated daily stats table
            con.execute("""
            CREATE TABLE daily_trip_stats AS
            SELECT
              pickup_date,
              COUNT(*) as trip_count,
              AVG(trip_distance) as avg_distance,
              AVG(fare_amount) as avg_fare,
              AVG(tip_amount) as avg_tip,
              SUM(total_amount) as total_revenue
            FROM yellow_trips_cleaned
            GROUP BY pickup_date
            ORDER BY pickup_date;
            """)
            
            print("Tables created successfully!")
            
            # Show row counts
            cleaned_count = con.execute("SELECT COUNT(*) FROM yellow_trips_cleaned").fetchone()[0]
            daily_count = con.execute("SELECT COUNT(*) FROM daily_trip_stats").fetchone()[0]
            
            print(f"yellow_trips_cleaned: {cleaned_count} rows")
            print(f"daily_trip_stats: {daily_count} rows")
            
            # Show daily stats sample
            print("\nDaily Trip Stats Sample:")
            print(con.execute("SELECT * FROM daily_trip_stats LIMIT 5").fetchdf())
            
        except Exception as e:
            print(f"Error creating transformed tables: {e}")
            print("Table structure might be different, please check the raw data schema")
    else:
        print("Transformed tables already exist")
        # Show sample data from transformed tables
        print("\nDaily Trip Stats Sample:")
        print(con.execute("SELECT * FROM daily_trip_stats LIMIT 5").fetchdf())
else:
    print("Raw data table not found. Please load the data first.")

con.close()

## Step 5: Orchestration and Scheduling

- Use Airflow, Prefect, or Kestra to automate ingestion, loading, and transformation.
- Example: Airflow DAG for monthly data ingestion and transformation.

In [ ]:
%%writefile airflow_example.py
# Example Airflow DAG for taxi data pipeline
# This would be placed in your Airflow DAGs folder

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import os
import duckdb

default_args = {
    'owner': 'data_engineer',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

def load_to_duckdb(file_path, table_name):
    """Load CSV data to DuckDB"""
    db_path = 'nyc_taxi.duckdb'
    con = duckdb.connect(db_path)
    con.execute(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM read_csv_auto('{file_path}');")
    con.close()
    return f"Loaded {file_path} to {table_name}"

def transform_data():
    """Transform raw taxi data"""
    db_path = 'nyc_taxi.duckdb'
    con = duckdb.connect(db_path)
    
    # Create cleaned table
    con.execute("""
    CREATE OR REPLACE TABLE yellow_trips_cleaned AS
    SELECT
      VendorID,
      tpep_pickup_datetime::TIMESTAMP as pickup_datetime,
      tpep_dropoff_datetime::TIMESTAMP as dropoff_datetime,
      passenger_count,
      trip_distance,
      RatecodeID,
      PULocationID as pickup_location_id,
      DOLocationID as dropoff_location_id,
      payment_type,
      fare_amount,
      tip_amount,
      total_amount
    FROM yellow_trips_raw
    WHERE trip_distance > 0 AND fare_amount > 0;
    """)
    
    # Create aggregated stats
    con.execute("""
    CREATE OR REPLACE TABLE daily_trip_stats AS
    SELECT
      DATE_TRUNC('day', pickup_datetime) as pickup_date,
      COUNT(*) as trip_count,
      AVG(trip_distance) as avg_distance,
      AVG(fare_amount) as avg_fare,
      SUM(total_amount) as total_revenue
    FROM yellow_trips_cleaned
    GROUP BY pickup_date
    ORDER BY pickup_date;
    """)
    
    con.close()
    return "Data transformation complete"

def export_to_parquet():
    """Export tables to Parquet format"""
    db_path = 'nyc_taxi.duckdb'
    con = duckdb.connect(db_path)
    
    # Export cleaned data
    con.execute("COPY (SELECT * FROM yellow_trips_cleaned) TO 'data/parquet/yellow_trips_cleaned.parquet' (FORMAT PARQUET);")
    
    # Export aggregated data
    con.execute("COPY (SELECT * FROM daily_trip_stats) TO 'data/parquet/daily_trip_stats.parquet' (FORMAT PARQUET);")
    
    con.close()
    return "Exported tables to Parquet format"

with DAG(
    'taxi_data_pipeline',
    default_args=default_args,
    description='Monthly taxi data processing pipeline',
    schedule_interval='@monthly',
    start_date=datetime(2021, 1, 1),
    catchup=False,
) as dag:
    
    # Create directory structure
    create_dirs = BashOperator(
        task_id='create_directories',
        bash_command='mkdir -p data/raw data/parquet'
    )
    
    # Download taxi data
    download_data = BashOperator(
        task_id='download_data',
        bash_command='''
            YEAR={{ execution_date.strftime('%Y') }}
            MONTH={{ execution_date.strftime('%m') }}
            URL="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_${YEAR}-${MONTH}.csv.gz"
            wget -nc "$URL" -P data/raw/
        '''
    )
    
    # Load data to DuckDB
    load_data = PythonOperator(
        task_id='load_to_duckdb',
        python_callable=load_to_duckdb,
        op_kwargs={
            'file_path': 'data/raw/yellow_tripdata_{{ execution_date.strftime("%Y-%m") }}.csv.gz',
            'table_name': 'yellow_trips_raw'
        }
    )
    
    # Transform data
    transform = PythonOperator(
        task_id='transform_data',
        python_callable=transform_data
    )
    
    # Export to Parquet
    export = PythonOperator(
        task_id='export_to_parquet',
        python_callable=export_to_parquet
    )
    
    # Define task dependencies
    create_dirs >> download_data >> load_data >> transform >> export

print("Example Airflow DAG has been written to airflow_example.py")
print("To use this, you would need to install Apache Airflow and place this file in your DAGs folder.")

## Step 6: Comparing Local-First and Cloud-First Environments

| Aspect | Local-First (DuckDB/MotherDuck) | Cloud-First (AWS/GCP/Azure) |
|--------|-------------------------------|-----------------------------|
| Data Size | Small to medium (<100GB) | Medium to huge (TBs+) |
| Performance | Fast for analytics, no network | Scales for big data, some latency |
| Cost | Lowest, free/cheap | Pay per query/storage |
| Complexity | Very low, no servers | Moderate, cloud setup |
| Use Cases | Local dev, prototyping, EDA | Production, multi-user, big data |
| Orchestration | Simple scripts, cron, Airflow | Cloud-native (Step Functions, Cloud Composer) |
| Maintenance | Manual updates | Managed services |
| Security | Limited, local files | IAM, encryption, VPC |

## Step 7: Portfolio Presentation Tips

- Organize your repo: data/, notebooks/, scripts/, dbt/, airflow_dags/, infrastructure/.
- Write a clear README with overview, architecture diagram, setup guide, results, and findings.
- Include visualizations (charts, maps) and discuss insights.
- Summarize pros/cons of each approach and suggest future work.
- Invite readers to reproduce your results and explore the project.

### Repository Organization Example

```
data_engineering_realtime_pipeline/
├── README.md                    # Project overview, setup guide, findings
├── pyproject.toml               # Dependencies and project configuration
├── docker-compose.yml           # Local infrastructure setup
├── mta_gtfs_realtime_prediction.ipynb  # GTFS-RT pipeline notebook
├── nyc_taxi_pipeline.ipynb      # NYC Taxi pipeline notebook
├── .env.example                 # Template for environment variables
├── data/                        # Data directory (git-ignored)
│   ├── raw/                     # Raw data files
│   └── parquet/                 # Processed Parquet files
├── dbt/                         # dbt models for transformations
├── airflow_dags/                # Airflow DAG definitions
├── scripts/                     # Utility scripts
│   ├── download_data.py         # Data download script
│   └── process_taxi_data.py     # Data processing script
├── flink_jobs/                  # Apache Flink job definitions
│   └── vehicle_metrics.py       # Flink job for processing vehicle data
└── trino/                       # Trino configuration
    └── catalog/                 # Catalog definitions
```

Below is a minimal, self-contained **producer** you can drop into your repo to poll the GTFS-Realtime feed on a configurable interval and push each update to Kafka.  

---
## 1. Update your `.env.example`
Add these new variables (so your local `.env` can override them):
```ini
# .env.example (add at the bottom)
KAFKA_BROKER=localhost:29092
KAFKA_TOPIC=vehicle_positions
POLL_INTERVAL=15      # seconds between pulls
MTA_FEED_URL=https://gtfsrt.prod.obanyc.com/vehiclePositions?key=${MTA_API_KEY}
```
---
## 2. Install the new dependency
Make sure your `pyproject.toml` or `requirements.txt` includes:
```
confluent-kafka
```
Then run:
```bash
pip install confluent-kafka
```
---
## 3. Add a streaming script
Create a file at `scripts/stream_gtfs.py`:
```python
#!/usr/bin/env python3
import os
import time
import logging
import requests
from google.protobuf import json_format
from google.transit import gtfs_realtime_pb2
from confluent_kafka import Producer
from dotenv import load_dotenv

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")

def main():
    load_dotenv()  # finds .env in project root
    api_url      = os.getenv("MTA_FEED_URL")
    broker       = os.getenv("KAFKA_BROKER")
    topic        = os.getenv("KAFKA_TOPIC", "vehicle_positions")
    interval_sec = int(os.getenv("POLL_INTERVAL", "15"))

    if not api_url or not broker:
        logging.error("Missing MTA_FEED_URL or KAFKA_BROKER in environment.")
        return

    # Configure Kafka producer
    producer = Producer({"bootstrap.servers": broker})
    logging.info(f"Starting GTFS-RT streamer → Kafka {broker} topic '{topic}' every {interval_sec}s")

    while True:
        try:
            resp = requests.get(api_url, timeout=10)
            resp.raise_for_status()
            feed = gtfs_realtime_pb2.FeedMessage()
            feed.ParseFromString(resp.content)

            batch = []
            for entity in feed.entity:
                # Convert each protobuf entity to a JSONable dict
                obj = json_format.MessageToDict(entity)
                batch.append(obj)

            if batch:
                for record in batch:
                    producer.produce(topic, key=record.get("id", ""), value=producer._serialize(record))
                producer.flush()
                logging.info(f"Published {len(batch)} records to '{topic}'")
            else:
                logging.info("No entities in feed this cycle")

        except Exception as e:
            logging.error(f"Error fetching/publishing GTFS-RT data: {e}")

        time.sleep(interval_sec)

if __name__ == "__main__":
    main()
```
**A few notes:**
1. It reads your **full feed URL** (including `?key=…`) from `MTA_FEED_URL`.  
2. It polls every `POLL_INTERVAL` seconds.  
3. It pushes each entity as a JSON message to the `vehicle_positions` topic.  
---
## 4. (Optional) Wire it into Docker Compose
If you’d rather run this as a containerized service, add to your `docker-compose.yml`:
```yaml
  gtfs-producer:
    build:
      context: .
      dockerfile: Dockerfile.producer
    env_file: .env
    depends_on:
      - kafka
    networks:
      - default
```
And create a `Dockerfile.producer`:
```dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY pyproject.toml poetry.lock* /app/
RUN pip install --no-cache-dir confluent-kafka python-dotenv google-protobuf requests
COPY scripts/stream_gtfs.py /app/
CMD ["python", "stream_gtfs.py"]
```
Then `docker-compose up -d gtfs-producer` will start your real-time stream alongside Kafka.
---
## 5. Run it locally
1. Fill out your `.env` from `.env.example`.  
2. Start Kafka (`docker-compose up -d zookeeper kafka`).  
3. In one terminal, run:
   ```bash
   python scripts/stream_gtfs.py
   ```
4. In another, consume from Kafka to verify:
   ```bash
   kafka-console-consumer --bootstrap-server localhost:29092 --topic vehicle_positions --from-beginning
   ```

That’s it—now you have continuous real-time ingestion of MTA BusTime GTFS-Realtime data into Kafka, ready for downstream Flink, Iceberg, Trino, Superset, or whatever comes next in your pipeline!

In [ ]:
%%writefile scripts/stream_gtfs.py
#!/usr/bin/env python3
import os
import time
import logging
import requests
from google.protobuf import json_format
from google.transit import gtfs_realtime_pb2
from confluent_kafka import Producer
from dotenv import load_dotenv

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")

def main():
    load_dotenv()  # finds .env in project root
    api_url      = os.getenv("MTA_FEED_URL")
    broker       = os.getenv("KAFKA_BROKER")
    topic        = os.getenv("KAFKA_TOPIC", "vehicle_positions")
    interval_sec = int(os.getenv("POLL_INTERVAL", "15"))

    if not api_url or not broker:
        logging.error("Missing MTA_FEED_URL or KAFKA_BROKER in environment.")
        return

    # Configure Kafka producer
    producer = Producer({"bootstrap.servers": broker})
    logging.info(f"Starting GTFS-RT streamer → Kafka {broker} topic '{topic}' every {interval_sec}s")

    while True:
        try:
            resp = requests.get(api_url, timeout=10)
            resp.raise_for_status()
            feed = gtfs_realtime_pb2.FeedMessage()
            feed.ParseFromString(resp.content)

            batch = []
            for entity in feed.entity:
                # Convert each protobuf entity to a JSONable dict
                obj = json_format.MessageToDict(entity)
                batch.append(obj)

            if batch:
                for record in batch:
                    producer.produce(topic, key=record.get("id", ""), value=producer._serialize(record))
                producer.flush()
                logging.info(f"Published {len(batch)} records to '{topic}'")
            else:
                logging.info("No entities in feed this cycle")

        except Exception as e:
            logging.error(f"Error fetching/publishing GTFS-RT data: {e}")

        time.sleep(interval_sec)

if __name__ == "__main__":
    main()